In [1]:
import pandas as pd
import numpy as np
import torch
from itertools import product

In [3]:
attr=pd.read_csv('20220308_monthlybond_sample.csv') #bond attributes
net=pd.read_csv('20220212_individual_holding_sample.csv') 

In [4]:
#preprocess for net
#step1 sort the cusip and fundid month
print('SORTED DATAFRAME')
net=net.sort_values(by = ['fundid', 'cusip','yq'], ascending = [True, True,True])

#calculate fund attributes (not use in this paper)
net['meanpar']=net.groupby(['fundid','yq'])['paramt'].transform(np.mean)
net['maxpar']=net.groupby(['fundid','yq'])['paramt'].transform(np.max)
net['minpar']=net.groupby(['fundid','yq'])['paramt'].transform(np.min)
net['stdpar']=net.groupby(['fundid','yq'])['paramt'].transform(np.std)

SORTED DATAFRAME


In [5]:
#fund attribute (not use in this paper)
fund_attr=net[['fundid','yq','holdingbond','totalpar','matchp','meanpar','maxpar','minpar','stdpar']]
#drop duplicates from DataFrame
fund_attr1 = fund_attr.drop_duplicates()

In [6]:
######try to calculate the return
#step1 sort the cusip and month
print('SORTED DATAFRAME')
attr=attr.sort_values(by = ['cusip', 'month'], ascending = [True, True])

SORTED DATAFRAME


In [7]:
#step 2 in order to calculate the time gap, make the attr['month'] with data type datetime
attr['month_datetime'] = pd.to_datetime(attr['month'], format='%Y%m')

In [8]:
month_gap_value = attr.groupby("cusip")["month_datetime"].apply(
    lambda x: (x-x.shift(1))/np.timedelta64(1, 'M')) 
attr['month_gap']=np.round(month_gap_value,0)

In [9]:
#step 3 check the price
attr['p'].describe()
#You might consider to remove all observations for a given bond if its average price is <30% (p<30) 
# or its average price is >150% (p>150). 

count    6132.000000
mean      107.410591
std        16.593793
min         7.125000
25%       100.390000
50%       107.426600
75%       114.815500
max       164.726000
Name: p, dtype: float64

In [10]:
df1 = attr.groupby('cusip').filter(lambda g: g['p'].mean() < 150)

In [11]:
df1['p'].describe()

count    6132.000000
mean      107.410591
std        16.593793
min         7.125000
25%       100.390000
50%       107.426600
75%       114.815500
max       164.726000
Name: p, dtype: float64

In [12]:
df2 = df1.groupby('cusip').filter(lambda g: g['p'].mean() > 30)

In [13]:
df2['p'].describe()

count    6132.000000
mean      107.410591
std        16.593793
min         7.125000
25%       100.390000
50%       107.426600
75%       114.815500
max       164.726000
Name: p, dtype: float64

In [14]:
q1=df2['p'].quantile(0.0075)
print(q1)
q2=df2['p'].quantile(0.9972)
print(q2)

43.586737500000005
159.3366048


In [15]:
df2['p'].clip(lower=df2['p'].quantile(0.0075), upper=df2['p'].quantile(0.9972), axis=0, inplace = True)

In [16]:
df2['p'].describe() #for price min 30 max 150 mean 102.58

count    6132.000000
mean      107.532865
std        15.977146
min        43.586738
25%       100.390000
50%       107.426600
75%       114.815500
max       159.336605
Name: p, dtype: float64

In [17]:
#step 4 check the yield
df2['yd'].describe()

count    5819.000000
mean        5.943992
std         6.753819
min         0.367411
25%         4.431515
50%         5.497463
75%         6.899289
max       453.709122
Name: yd, dtype: float64

In [18]:
#For bond yields (yd) which is in %, there will be outliers on one large side. 
# You might treat yd as missing if yd>30 (about 0.5%) or yd>20 (about 1%). 
q1=df2['yd'].quantile(0.991)
q1


17.343087599999976

In [19]:
df2['yd'].clip(upper=df2['yd'].quantile(0.991), axis=0, inplace = True)

In [20]:
df2['yd'].describe() #for yiled, min 0 max 28.2 mean 5.3

count    5819.000000
mean        5.780807
std         2.680529
min         0.367411
25%         4.431515
50%         5.497463
75%         6.899289
max        17.343088
Name: yd, dtype: float64

In [21]:
#step 5 calculate the return
df2['return_term']=df2['p']*10+df2['ai']+df2['cpn']

#for each bond calculate the return
return_value = df2.groupby("cusip")["return_term"].apply(
    lambda x: (x-x.shift(1))/x.shift(1)) 
df2['return']=return_value

#devided by the month_gap
df2['return']=df2['return']/df2['month_gap']

# if the month gap large than 12, set the return as nan
df2.loc[df2['month_gap']>12,'return']=np.NaN

In [22]:
df2['return'].describe()

count    6074.000000
mean        0.002096
std         0.046677
min        -0.422279
25%        -0.014574
50%         0.002225
75%         0.016765
max         0.880855
Name: return, dtype: float64

In [23]:
#winsorize on the return
q1=df2['return'].quantile(0.01)
print(q1)
q2=df2['return'].quantile(0.995)
print(q2)

df2['return'].clip(lower=df2['return'].quantile(0.01), upper=df2['return'].quantile(0.995), axis=0, inplace = True)

-0.13065298023989488
0.19204515438235426


In [24]:
df2['return'].describe() #min -0.122 max 0.198 mean 0.00188

count    6074.000000
mean        0.002081
std         0.039048
min        -0.130653
25%        -0.014574
50%         0.002225
75%         0.016765
max         0.192045
Name: return, dtype: float64

In [25]:
attr=df2.copy()

In [26]:
#data preprocess
#net: 1. remove the bond and fund with only a few records
net=net.groupby('cusip').filter(lambda x: x['fundid'].nunique() > 20)
#net=net.groupby('fundid').filter(lambda x: x['cusip'].nunique() > 20)
net=net[['cusip','fundid','yq','paramt']]

net_bond=net['cusip'].unique()
net_yq=net['yq'].unique()

In [27]:
#attr
#1 for all cusip fill the month
df1=pd.DataFrame(data=product(attr['cusip'].unique(), np.sort(attr['month'].unique())))
df1.columns=['cusip','month']
attr=pd.merge(df1, attr, how='left', on=['cusip','month'])
attr=attr.sort_values(['cusip','month'])

In [28]:
#attr 3 fill the missing in the rating
#fill the missing by the most recent rating 
attr['rat_moodys']=attr.groupby('cusip')['rat_moodys'].apply(lambda x: x.fillna(method='ffill'))
attr['rat_low']=attr.groupby('cusip')['rat_low'].apply(lambda x: x.fillna(method='ffill'))
attr['rat_med']=attr.groupby('cusip')['rat_med'].apply(lambda x: x.fillna(method='ffill'))
attr['rat_last']=attr.groupby('cusip')['rat_last'].apply(lambda x: x.fillna(method='ffill'))
# fill the missing in the ttm
attr['ttm']=attr.groupby('cusip')['ttm'].apply(lambda x: x.fillna(method='ffill'))
# fill the missing in the cpn
attr['cpn']=attr.groupby('cusip')['cpn'].apply(lambda x: x.fillna(method='ffill'))
#fill the missing in ai
attr['ai']=attr.groupby('cusip')['ai'].apply(lambda x: x.fillna(method='ffill'))

In [29]:
#attr: 2.select the bond and yq in the net dataset
attr=attr[attr['cusip'].isin(net_bond)] #bond in net

#yq in net
#transfer the month to yq in attr data

#1match the month in bond attribute with the quarter in net
attr['yy']=attr['month']//100*100
attr['mm']=attr['month']%100
attr['yq']=1
attr.loc[attr['mm'].isin((1,2,3)),'yq']=attr[attr['mm'].isin((1,2,3))]['yy'].values+1
attr.loc[attr['mm'].isin((4,5,6)),'yq']=attr[attr['mm'].isin((4,5,6))]['yy'].values+2
attr.loc[attr['mm'].isin((7,8,9)),'yq']=attr[attr['mm'].isin((7,8,9))]['yy'].values+3
attr.loc[attr['mm'].isin((10,11,12)),'yq']=attr[attr['mm'].isin((10,11,12))]['yy'].values+4
#select yq in the net dataset
attr=attr[attr['yq'].isin(net_yq)] #bond in net

In [30]:
#attr 5 keep the bond with at least 10 months records of 7 attributes and targets
select_attr=['p', 'yd', 'return','rat_moodys', 'rat_low', 'rat_med','rat_last','ttm','cpn','ai']
remove_bond=np.array([])
for var in select_attr:
    remove_item=attr['cusip'].unique()[attr.groupby('cusip')[var].count()<10]
    remove_bond=np.append(remove_bond,remove_item)

remove_bond=np.unique(remove_bond)
print(len(remove_bond),'bonds will be removed')

attr=attr[~attr['cusip'].isin(remove_bond)]


1 bonds will be removed


In [31]:
#merge
#1.select common bond and yq in attr and net
#common bond
print('number of bond in bond attributes',len(attr['cusip'].unique()))
print('number of bond in bond fund bipartite graph',len(net['cusip'].unique()))
common_bond=np.intersect1d(attr['cusip'].unique(),net['cusip'].unique())
print(len(common_bond))

#common quarter
print('number of yq in bond attributes',len(attr['yq'].unique()))
print('number of yq in bond fund bipartite graph',len(net['yq'].unique()))
common_yq=np.intersect1d(attr['yq'].unique(),net['yq'].unique())
print(len(common_yq))
common_yq=np.sort(common_yq)
#2. decide the sequence and net tensor
#3.decide the sequence and net tensor

number of bond in bond attributes 54
number of bond in bond fund bipartite graph 55
54
number of yq in bond attributes 58
number of yq in bond fund bipartite graph 73
58


In [32]:
#select common in the bond attribute dataset
attr=attr[attr['cusip'].isin(common_bond) & attr['yq'].isin(common_yq)]
attr=attr[['cusip','month','yq','rat_moodys','rat_low','rat_med','rat_last','ttm','cpn','ai','p','yd','return']]

#select common bond and common yq in net
net=net[['cusip','yq','fundid','paramt']]
net=net[net['cusip'].isin(common_bond) & net['yq'].isin(common_yq)]

In [33]:
#merge the fund_attr
#attr
#1 for all cusip fill the month
#common_fund=net['fundid'].unique()
df1=pd.DataFrame(data=product(fund_attr1['fundid'].unique(), np.sort(fund_attr1['yq'].unique())))
df1.columns=['fundid','yq']
df1.shape
fund_attr2=pd.merge(df1, fund_attr1, how='left', on=['fundid','yq'])
fund_attr2=fund_attr2.sort_values(['fundid','yq'])

In [34]:
fund_attr=fund_attr2.copy()

In [35]:
#attr 3 fill the missing in the rating
#fill the missing by the most recent rating 
fund_attr['holdingbond']=fund_attr.groupby('fundid')['holdingbond'].apply(lambda x: x.fillna(method='ffill'))
fund_attr['totalpar']=fund_attr.groupby('fundid')['totalpar'].apply(lambda x: x.fillna(method='ffill'))
fund_attr['matchp']=fund_attr.groupby('fundid')['matchp'].apply(lambda x: x.fillna(method='ffill'))
fund_attr['meanpar']=fund_attr.groupby('fundid')['meanpar'].apply(lambda x: x.fillna(method='ffill'))
fund_attr['maxpar']=fund_attr.groupby('fundid')['maxpar'].apply(lambda x: x.fillna(method='ffill'))
fund_attr['minpar']=fund_attr.groupby('fundid')['minpar'].apply(lambda x: x.fillna(method='ffill'))
fund_attr['stdpar']=fund_attr.groupby('fundid')['stdpar'].apply(lambda x: x.fillna(method='ffill'))

In [36]:
#merge the fund_attr
#attr
#1 for all cusip fill the month
common_fund=net['fundid'].unique()
df1=pd.DataFrame(data=product(common_fund, common_yq))
df1.columns=['fundid','yq']
fund_attr=df1.merge(fund_attr, how='left', on=['fundid','yq'])
#fund_attr=fund_attr.sort_values(['fundid','yq'])

In [37]:
attr.to_csv('attr_clean_new.csv')
net.to_csv('net_clean_new.csv')
fund_attr.to_csv('fund_clean_new.csv')